In [19]:

!git add PredictNHL.ipynb


!git commit -m "Add PredictNHL.ipynb"


!git push origin main


On branch main
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.cache/
	.gitconfig
	.ipynb_checkpoints/
	.ipython/
	.jupyter/
	.keras/
	.matplotlib/
	.viminfo
	.vscode/
	.wdm/
	3D Objects/
	AppData/
	Basic Cluster.txt
	Cluster.ipynb
	Clustering/
	Contacts/
	Creative Cloud Files Personal Account NIC.26@HOTMAIL.COM 695E445F62B8CBFC0A495F8A@AdobeID/
	Desktop/
	Documents/
	Downloads/
	Favorites/
	HiveOSInstaller/
	HltvData/
	Languages/
	Links/
	MicrosoftEdgeBackups/
	Muse Hub/
	Music/
	NTUSER.DAT
	NTUSER.DAT{2ad838bc-efea-11ee-a54d-000d3a94eaa1}.TM.blf
	NTUSER.DAT{2ad838bc-efea-11ee-a54d-000d3a94eaa1}.TMContainer00000000000000000001.regtrans-ms
	NTUSER.DAT{2ad838bc-efea-11ee-a54d-000d3a94eaa1}.TMContainer00000000000000000002.regtrans-ms
	OneDrive/
	Pictures/
	SHK_Bot_Installer_And_Uninstaller.exe
	Saved Games/
	Searches/
	StrongholdKingdoms.exe
	UserBenchmark/
	Videos/
	correlation_matrix_selected_features.csv
	decision_tree
	decision_tree.pdf
	enhanced_g

To https://github.com/nic01as1/NHLPredictor.git
 ! [rejected]        main -> main (fetch first)
error: failed to push some refs to 'https://github.com/nic01as1/NHLPredictor.git'
hint: Updates were rejected because the remote contains work that you do not
hint: have locally. This is usually caused by another repository pushing to
hint: the same ref. If you want to integrate the remote changes, use
hint: 'git pull' before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


In [ ]:
#if stuck :
!git push origin main --force


In [1]:
!pip install tensorflow
!pip install xgboost
!pip install streamlit
!pip install imbalanced-learn
!pip install shap
!pip install statsmodels



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import time
import warnings
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (accuracy_score, log_loss, roc_auc_score, f1_score, classification_report, make_scorer)
from sklearn.inspection import permutation_importance
from imblearn.over_sampling import SMOTE

In [3]:
#AJOUTER LE NOMBRE DE JOURS DE REPOS ! et ajouter 2 autres mod`les (Neural et Logistic)

In [4]:
#Retrieve the rank of each teams

url = 'https://www.hockey-reference.com/leagues/NHL_2025.html#all_stats'

with webdriver.Chrome() as driver:
    driver.get(url)
    time.sleep(2) 
    soup = BeautifulSoup(driver.page_source, 'html.parser')

driver.quit()

table = soup.find('table', id='stats')

if table is None:
    print("Team Statistics table not found")
else:
    
    rows = table.find_all('tr')[1:] 
    teams = [
        row.find_all('td')[0].text.strip()
        for row in rows
        if row.find_all('td')
    ]
    ranks = list(range(1, len(teams) + 1)) 

    
    df_rank = pd.DataFrame({
        'Team': teams,
        'Rank': ranks,
        'Team Rank': [f"{team} {rank}" for team, rank in zip(teams, ranks)]
    })
    print(df_rank)
    

                     Team  Rank                 Team Rank
0           Winnipeg Jets     1           Winnipeg Jets 1
1     Washington Capitals     2     Washington Capitals 2
2         Edmonton Oilers     3         Edmonton Oilers 3
3            Dallas Stars     4            Dallas Stars 4
4     Carolina Hurricanes     5     Carolina Hurricanes 5
5    Vegas Golden Knights     6    Vegas Golden Knights 6
6        Florida Panthers     7        Florida Panthers 7
7     Toronto Maple Leafs     8     Toronto Maple Leafs 8
8          Minnesota Wild     9          Minnesota Wild 9
9       New Jersey Devils    10      New Jersey Devils 10
10     Colorado Avalanche    11     Colorado Avalanche 11
11      Los Angeles Kings    12      Los Angeles Kings 12
12        Ottawa Senators    13        Ottawa Senators 13
13      Detroit Red Wings    14      Detroit Red Wings 14
14          Boston Bruins    15          Boston Bruins 15
15    Tampa Bay Lightning    16    Tampa Bay Lightning 16
16         Cal

In [5]:

#Retrieve the infos of EACH games for the season

def scrape_game_history(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table")
    if table is None:
        print(f"Table not found on {url}")
        return pd.DataFrame()  #

    
    data = {
        "Date": [],
        "Home Team": [],
        "Home Score": [],
        "Away Team": [],
        "Away Score": [],
        "Result": [],
        "Home Win": []
    }

    
    for row in table.find_all("tr")[1:]:  # Skip the header
        columns = row.find_all("td")
        if len(columns) < 6:  
            continue

        
        date = columns[1].text.strip()
        home_team = columns[3].text.strip()
        away_team = columns[4].text.strip()
        result = columns[5].text.strip()

        
        if "-" in result:  
            scores = result.split("-")
            if len(scores) == 2 and scores[0].strip().isdigit() and scores[1].strip().isdigit():
                home_score = int(scores[0].strip())
                away_score = int(scores[1].strip())
                home_win = home_score > away_score
            else:  
                home_score = away_score = home_win = None
        else:
            home_score = away_score = home_win = None

        
        data["Date"].append(date)
        data["Home Team"].append(home_team)
        data["Away Team"].append(away_team)
        data["Result"].append(result)
        data["Home Score"].append(home_score)
        data["Away Score"].append(away_score)
        data["Home Win"].append(home_win)

    
    df_game = pd.DataFrame(data)
    df_game["Date"] = df_game["Date"].apply(lambda x: x.split(" ")[0])  # Extract just the date
    return df_game



url_history = "https://fixturedownload.com/results/nhl-2024"


df_game = scrape_game_history(url_history)


if df_game.empty:
    print("No game data found.")
else:
    print(f"Scraped {len(df_game)} games.")
    print(df_game)

Scraped 1312 games.
            Date              Home Team  Home Score            Away Team  \
0     04/10/2024         Buffalo Sabres         1.0    New Jersey Devils   
1     05/10/2024      New Jersey Devils         3.0       Buffalo Sabres   
2     08/10/2024         Seattle Kraken         2.0      St. Louis Blues   
3     08/10/2024       Florida Panthers         6.0        Boston Bruins   
4     09/10/2024       Utah Hockey Club         5.0   Chicago Blackhawks   
...          ...                    ...         ...                  ...   
1307  17/04/2025        Ottawa Senators         NaN  Carolina Hurricanes   
1308  17/04/2025       New York Rangers         NaN  Tampa Bay Lightning   
1309  17/04/2025    Pittsburgh Penguins         NaN  Washington Capitals   
1310  17/04/2025  Columbus Blue Jackets         NaN   New York Islanders   
1311  18/04/2025      Los Angeles Kings         NaN       Calgary Flames   

      Away Score Result Home Win  
0            4.0  1 - 4    False

In [6]:
#Normalise Both datasource
team_name_mapping = {
    "Montréal Canadiens": "Montreal Canadiens", 
    
}


df_game["Home Team"] = df_game["Home Team"].replace(team_name_mapping)
df_game["Away Team"] = df_game["Away Team"].replace(team_name_mapping)


In [7]:
# Merge df_rank to add Home Team Rank
df_game = df_game.merge(
    df_rank.rename(columns={"Team": "Home Team", "Rank": "Home Team Rank"}), 
    on="Home Team", 
    how="left"
)

# Merge df_rank to add Away Team Rank
df_game = df_game.merge(
    df_rank.rename(columns={"Team": "Away Team", "Rank": "Away Team Rank"}), 
    on="Away Team", 
    how="left"
)

# Reorder the final columns
final_columns = [
    "Date", "Home Team", "Home Score", "Away Team", "Away Score",
    "Home Team Rank", "Away Team Rank", "Home Win"
]
df_final = df_game.loc[:, final_columns] 

print(df_final)


            Date              Home Team  Home Score            Away Team  \
0     04/10/2024         Buffalo Sabres         1.0    New Jersey Devils   
1     05/10/2024      New Jersey Devils         3.0       Buffalo Sabres   
2     08/10/2024         Seattle Kraken         2.0      St. Louis Blues   
3     08/10/2024       Florida Panthers         6.0        Boston Bruins   
4     09/10/2024       Utah Hockey Club         5.0   Chicago Blackhawks   
...          ...                    ...         ...                  ...   
1307  17/04/2025        Ottawa Senators         NaN  Carolina Hurricanes   
1308  17/04/2025       New York Rangers         NaN  Tampa Bay Lightning   
1309  17/04/2025    Pittsburgh Penguins         NaN  Washington Capitals   
1310  17/04/2025  Columbus Blue Jackets         NaN   New York Islanders   
1311  18/04/2025      Los Angeles Kings         NaN       Calgary Flames   

      Away Score  Home Team Rank  Away Team Rank Home Win  
0            4.0           

In [8]:
#DATA MANIPULATION


# Filter out the Unplayed Games
df_final = df_final.dropna(subset=["Home Score", "Away Score"])

# Convert Scores and Dates to Appropriate Data Types
df_final["Date"] = pd.to_datetime(df_final["Date"], format="%d/%m/%Y")
df_final[["Home Score", "Away Score"]] = df_final[["Home Score", "Away Score"]].astype(int)
df_final["Home Win"] = df_final["Home Win"].astype(bool)

# Sort Data by Date for Chronological Calculations
df_final = df_final.sort_values(by="Date").reset_index(drop=True)

# Function: Calculate Last 10 Games Stats
def calculate_last_10_stats(df, team_column):
    last_10_wins = []
    team_games = {team: [] for team in pd.concat([df["Home Team"], df["Away Team"]]).unique()}

    for _, row in df.iterrows():
        team = row[team_column]
        recent_games = team_games[team][-10:]  # Last 10 games
        last_10_wins.append(sum(recent_games))
        team_games[row["Home Team"]].append(row["Home Win"])
        team_games[row["Away Team"]].append(not row["Home Win"])
    return last_10_wins

# Add Last 10 Wins for Home and Away Teams
df_final["Home Last 10 Wins"] = calculate_last_10_stats(df_final, "Home Team")
df_final["Away Last 10 Wins"] = calculate_last_10_stats(df_final, "Away Team")

# Add Whether Teams Played Yesterday
def calculate_played_yesterday(df, team_column):
    played_yesterday = []
    last_game_date = {}

    for _, row in df.iterrows():
        team = row[team_column]
        played_yesterday.append(last_game_date.get(team) == row["Date"] - pd.Timedelta(days=1))
        last_game_date[team] = row["Date"]
    return played_yesterday

df_final["Home Played Yesterday"] = calculate_played_yesterday(df_final, "Home Team")
df_final["Away Played Yesterday"] = calculate_played_yesterday(df_final, "Away Team")

# Add Win Rate for Home and Away Teams
def calculate_win_rate(df, team_column, is_home_column):
    win_rate = []
    team_stats = {team: {"wins": 0, "games": 0} for team in pd.concat([df["Home Team"], df["Away Team"]]).unique()}

    for _, row in df.iterrows():
        team = row[team_column]
        stats = team_stats[team]
        win_rate.append(stats["wins"] / stats["games"] if stats["games"] > 0 else 0)
        
        
        if is_home_column:
            stats["wins"] += row["Home Win"]
        else:
            stats["wins"] += not row["Home Win"]
        stats["games"] += 1
    return win_rate

df_final["Home Win Rate"] = calculate_win_rate(df_final, "Home Team", is_home_column=True)
df_final["Away Win Rate"] = calculate_win_rate(df_final, "Away Team", is_home_column=False)

# Add Overall Win Streak for Home and Away Teams
def calculate_overall_win_streak(df):
    streak = {}
    home_streaks = []
    away_streaks = []

    for _, row in df.iterrows():
        home_team = row["Home Team"]
        away_team = row["Away Team"]

        
        home_streaks.append(streak.get(home_team, 0))
        away_streaks.append(streak.get(away_team, 0))

        
        if row["Home Win"]:
            streak[home_team] = streak.get(home_team, 0) + 1
            streak[away_team] = 0
        else:
            streak[home_team] = 0
            streak[away_team] = streak.get(away_team, 0) + 1
    return home_streaks, away_streaks

home_streaks, away_streaks = calculate_overall_win_streak(df_final)
df_final["Home Team Overall Win Streak Before Game"] = home_streaks
df_final["Away Team Overall Win Streak Before Game"] = away_streaks

# Add Average Opponent Strength for Home and Away Teams
def calculate_avg_opponent_rank(df, team_column, opponent_rank_column):
    avg_opponent_rank = []
    opponent_stats = {team: [] for team in pd.concat([df["Home Team"], df["Away Team"]]).unique()}

    for _, row in df.iterrows():
        team = row[team_column]
        avg_opponent_rank.append(
            np.mean(opponent_stats[team][-10:]) if opponent_stats[team] else np.nan
        )
        opponent_stats[row["Home Team"]].append(row["Away Team Rank"])
        opponent_stats[row["Away Team"]].append(row["Home Team Rank"])
    return avg_opponent_rank

df_final["Home Opponent Strength"] = calculate_avg_opponent_rank(df_final, "Home Team", "Away Team Rank")
df_final["Away Opponent Strength"] = calculate_avg_opponent_rank(df_final, "Away Team", "Home Team Rank")

def calculate_days_since_last_game(df):
    last_game_date = {}  # Dictionary to track the last game date for each team
    home_days_since_last_game = []
    away_days_since_last_game = []

    for _, row in df.iterrows():
        home_team = row["Home Team"]
        away_team = row["Away Team"]
        current_date = row["Date"]

        # Calculate days since last game for the home team
        if home_team in last_game_date:
            days_since_last_home_game = (current_date - last_game_date[home_team]).days
        else:
            days_since_last_home_game = None  
        home_days_since_last_game.append(days_since_last_home_game)

        # Calculate days since last game for the away team
        if away_team in last_game_date:
            days_since_last_away_game = (current_date - last_game_date[away_team]).days
        else:
            days_since_last_away_game = None  
        away_days_since_last_game.append(days_since_last_away_game)

        # Update the last game date for both teams
        last_game_date[home_team] = current_date
        last_game_date[away_team] = current_date

    return home_days_since_last_game, away_days_since_last_game


# Apply the function to the DataFrame
home_days, away_days = calculate_days_since_last_game(df_final)


df_final["Home Rest Days Since Last Game"] = home_days
df_final["Away Rest Days Since Last Game"] = away_days

print(df_final[["Date", "Home Team", "Away Team", "Home Opponent Strength", "Away Opponent Strength","Home Last 10 Wins","Away Last 10 Wins","Home Played Yesterday","Away Played Yesterday","Home Win Rate","Away Win Rate","Home Team Overall Win Streak Before Game","Away Team Overall Win Streak Before Game","Away Rest Days Since Last Game","Home Rest Days Since Last Game"]].tail(10))


df_final.to_csv("enhanced_game_data.csv", index=False)


          Date            Home Team              Away Team  \
847 2025-02-05        Winnipeg Jets    Carolina Hurricanes   
848 2025-02-05      St. Louis Blues        Edmonton Oilers   
849 2025-02-05       Buffalo Sabres  Columbus Blue Jackets   
850 2025-02-05  Washington Capitals       Florida Panthers   
851 2025-02-05  Pittsburgh Penguins      New Jersey Devils   
852 2025-02-05  Tampa Bay Lightning        Ottawa Senators   
853 2025-02-05        Boston Bruins         Minnesota Wild   
854 2025-02-05        Anaheim Ducks           Dallas Stars   
855 2025-02-05   New York Islanders   Vegas Golden Knights   
856 2025-02-05      San Jose Sharks     Montreal Canadiens   

     Home Opponent Strength  Away Opponent Strength  Home Last 10 Wins  \
847                    18.2                    20.0                  8   
848                    15.0                    14.9                  4   
849                    17.8                    15.7                  5   
850                  

In [9]:

results1 = df_final
results1.tail(10)

,Date,Home Team,Home Score,Away Team,Away Score,Home Team Rank,Away Team Rank,Home Win,Home Last 10 Wins,Away Last 10 Wins,Home Played Yesterday,Away Played Yesterday,Home Win Rate,Away Win Rate,Home Team Overall Win Streak Before Game,Away Team Overall Win Streak Before Game,Home Opponent Strength,Away Opponent Strength,Home Rest Days Since Last Game,Away Rest Days Since Last Game
847,2025-02-05,Winnipeg Jets,3,Carolina Hurricanes,0,1,5,True,8,6,False,False,0.714286,0.461538,6,0,18.2,20.0,3.0,3.0
848,2025-02-05,St. Louis Blues,2,Edmonton Oilers,3,25,3,False,4,6,False,False,0.440000,0.608696,1,0,15.0,14.9,2.0,3.0
849,2025-02-05,Buffalo Sabres,3,Columbus Blue Jackets,2,29,19,True,5,6,False,False,0.444444,0.321429,3,0,17.8,15.7,3.0,3.0
850,2025-02-05,Washington Capitals,6,Florida Panthers,3,2,7,True,7,7,False,False,0.640000,0.592593,0,3,17.4,19.1,3.0,3.0
851,2025-02-05,Pittsburgh Penguins,2,New Jersey Devils,3,24,10,False,4,4,False,False,0.500000,0.518519,2,0,23.0,17.9,3.0,3.0
852,2025-02-05,Tampa Bay Lightning,4,Ottawa Senators,3,16,13,True,4,7,False,True,0.640000,0.464286,0,5,19.4,13.8,3.0,1.0
853,2025-02-05,Boston Bruins,3,Minnesota Wild,0,15,9,True,6,4,False,False,0.607143,0.689655,1,0,15.3,16.7,4.0,3.0
854,2025-02-05,Anaheim Ducks,2,Dallas Stars,1,27,4,True,4,7,False,False,0.461538,0.583333,1,5,17.6,13.4,3.0,3.0
855,2025-02-05,New York Islanders,2,Vegas Golden Knights,1,20,6,True,7,2,False,False,0.480000,0.480000,1,0,17.2,17.1,3.0,3.0
856,2025-02-05,San Jose Sharks,3,Montreal Canadiens,4,32,22,False,2,4,False,False,0.346154,0.440000,0,0,19.6,13.6,5.0,3.0


In [10]:

scorer = make_scorer(f1_score, pos_label=None, average='weighted')

# Define y_true based on the Home Win column
y_true = df_final["Home Win"].astype(int)  # Convert True/False to 1/0

# Create a dummy prediction where all predictions are 1 (Home Win = True)
y_pred = [1] * len(y_true)

# Calculate the weighted F1 score
f1 = f1_score(y_true, y_pred, pos_label=None, average="weighted")

# Print the F1 score
print(f"F1 Score: {f1:.4f}")

F1 Score: 0.3885


In [11]:

#Target and features
target = "Home Win"
features = [
    "Home Opponent Strength", 
    "Away Opponent Strength",
    "Home Team Rank",
    "Away Team Rank",
    "Home Last 10 Wins",
    "Away Last 10 Wins",
    "Home Played Yesterday",
    "Away Played Yesterday",
    "Home Win Rate",
    "Away Win Rate",
    "Home Team Overall Win Streak Before Game",
    "Away Team Overall Win Streak Before Game",
    "Away Rest Days Since Last Game",
    "Home Rest Days Since Last Game"
]


X = df_final[features]
y = df_final[target].astype(int)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

#Param_Grid
param_grid_rf = {
    "n_estimators": [50, 100, 200],
    "max_depth": [5, 10, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

param_grid_dt = {
    "max_depth": [5, 10, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}


rf = RandomForestClassifier(random_state=42)
dt = DecisionTreeClassifier(random_state=42)


grid_search_rf = GridSearchCV(rf, param_grid_rf, scoring="f1_weighted", cv=5, n_jobs=-1)
grid_search_rf.fit(X_train, y_train)
best_rf = grid_search_rf.best_estimator_
print(f"Best Random Forest Params: {grid_search_rf.best_params_}")


grid_search_dt = GridSearchCV(dt, param_grid_dt, scoring="f1_weighted", cv=5, n_jobs=-1)
grid_search_dt.fit(X_train, y_train)
best_dt = grid_search_dt.best_estimator_
print(f"Best Decision Tree Params: {grid_search_dt.best_params_}")


models = {
    "Random Forest": best_rf,
    "Decision Tree": best_dt
}


results = []

#Loop Train of models
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average="weighted")
    results.append({"Model": model_name, "F1 Score": f1})

    # Feature importance
    if hasattr(model, "feature_importances_"):
        feature_importance_df = pd.DataFrame({
            "Feature": features,
            "Importance": model.feature_importances_
        }).sort_values(by="Importance", ascending=False)
        
        print(f"\nFeature Importance ({model_name}):")
        print(feature_importance_df)
        feature_importance_df.to_csv(f"feature_importance_{model_name.lower().replace(' ', '_')}.csv", index=False)

    #PErmutation
    perm_importance = permutation_importance(model, X_test, y_test, scoring="f1_weighted", random_state=42)
    perm_importance_df = pd.DataFrame({
        "Feature": features,
        "Importance": perm_importance.importances_mean
    }).sort_values(by="Importance", ascending=False)

    print(f"\nPermutation Importance ({model_name}):")
    print(perm_importance_df)
    perm_importance_df.to_csv(f"feature_importance_permutation_{model_name.lower().replace(' ', '_')}.csv", index=False)


results_df = pd.DataFrame(results).sort_values(by="F1 Score", ascending=False)
print("\nModel Performance Summary:")
print(results_df)

# Save model performance results
results_df.to_csv("model_performance_comparison.csv", index=False)

Best Random Forest Params: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best Decision Tree Params: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2}

Feature Importance (Random Forest):
                                     Feature  Importance
2                             Home Team Rank    0.167162
3                             Away Team Rank    0.146112
1                     Away Opponent Strength    0.107678
0                     Home Opponent Strength    0.107239
8                              Home Win Rate    0.093603
9                              Away Win Rate    0.093226
12            Away Rest Days Since Last Game    0.068926
5                          Away Last 10 Wins    0.044903
13            Home Rest Days Since Last Game    0.042223
4                          Home Last 10 Wins    0.036438
10  Home Team Overall Win Streak Before Game    0.036385
11  Away Team Overall Win Streak Before Game    0.027076
7                  

In [12]:
# Retain only the important features with positive permutation importance for RF which is the best model
selected_features = [
    "Away Team Rank", "Home Team Rank",  
    "Away Win Rate", "Home Team Overall Win Streak Before Game", 
    "Home Opponent Strength", "Away Opponent Strength", 
    "Home Last 10 Wins", "Away Played Yesterday", 
    "Home Rest Days Since Last Game", "Away Last 10 Wins"
]



In [13]:
# Initialize Label Encoder
encoder = LabelEncoder()


df_final["Home Team Encoded"] = encoder.fit_transform(df_final["Home Team"])
df_final["Away Team Encoded"] = encoder.transform(df_final["Away Team"])

print(df_final[["Home Team", "Home Team Encoded"]].head())
print(df_final[["Away Team", "Away Team Encoded"]].head())


encoder.fit(results1["Home Team"].values)

home_teams = encoder.transform(results1["Home Team"].values)
visitor_teams = encoder.transform(results1["Away Team"].values)


X_teams = np.vstack([home_teams, visitor_teams]).T
print(X_teams[:5], X_teams.shape)  # Check first few rows & dimensions


onehot = OneHotEncoder()
X_teams = onehot.fit_transform(X_teams).toarray()

print(X_teams[:5]) 
print(X_teams.shape)  

           Home Team  Home Team Encoded
0     Buffalo Sabres                  2
1  New Jersey Devils                 16
2     Seattle Kraken                 23
3   Florida Panthers                 11
4   Utah Hockey Club                 27
            Away Team  Away Team Encoded
0   New Jersey Devils                 16
1      Buffalo Sabres                  2
2     St. Louis Blues                 24
3       Boston Bruins                  1
4  Chicago Blackhawks                  5
[[ 2 16]
 [16  2]
 [23 24]
 [11  1]
 [27  5]] (857, 2)
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [14]:


X_homehigher = df_final[selected_features].values 
X_all = np.hstack([X_homehigher, X_teams])
print("Feature matrix shape:", X_all.shape)


Feature matrix shape: (857, 74)


In [15]:
X_homehigher = df_final[selected_features].values 

print("Shape of Numerical Features (X_homehigher):", X_homehigher.shape)
print("Shape of One-Hot Encoded Teams (X_teams):", X_teams.shape)


X_all = np.hstack([X_homehigher, X_teams])
print(" Final Feature Matrix Shape (X_all):", X_all.shape)


sample_data = pd.DataFrame(X_all)


print("\n Preview of last 5 observations:")
print(sample_data.tail())



Shape of Numerical Features (X_homehigher): (857, 10)
Shape of One-Hot Encoded Teams (X_teams): (857, 64)
 Final Feature Matrix Shape (X_all): (857, 74)

 Preview of last 5 observations:
     0   1         2  3     4     5  6      7    8  9   ...   64   65   66  \
852  13  16  0.464286  0  19.4  13.8  4   True  3.0  7  ...  0.0  0.0  0.0   
853   9  15  0.689655  1  15.3  16.7  6  False  4.0  4  ...  0.0  0.0  0.0   
854   4  27  0.583333  1  17.6  13.4  4  False  3.0  7  ...  0.0  0.0  0.0   
855   6  20      0.48  1  17.2  17.1  7  False  3.0  2  ...  0.0  0.0  0.0   
856  22  32      0.44  0  19.6  13.6  2  False  5.0  4  ...  0.0  0.0  0.0   

      67   68   69   70   71   72   73  
852  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
853  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
854  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
855  0.0  0.0  0.0  0.0  1.0  0.0  0.0  
856  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 74 columns]


In [16]:

param_grid_rf = {
    "max_features": ['sqrt', 'log2', 0.1, 0.3, None],
    "n_estimators": [100, 200, 300, 500],
    "criterion": ["gini", "entropy"],
    "min_samples_leaf": [1, 2, 4, 6, 8],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "bootstrap": [True, False]
}


scorer = make_scorer(f1_score, average="weighted")


rf_model = RandomForestClassifier(random_state=10)


print("\nTraining Random Forest with Grid Search...")
grid_rf = GridSearchCV(rf_model, param_grid_rf, scoring=scorer, error_score="raise", cv=5, n_jobs=-1)


try:
    grid_rf.fit(X_all, y_true)  
    best_rf_model = grid_rf.best_estimator_  

    print("\nBest Random Forest Model:")
    print(f"   - Best F1 Score: {grid_rf.best_score_:.4f}")
    print(f"   - Best Parameters: {grid_rf.best_params_}")

except Exception as e:
    print(f"Error during Grid Search for Random Forest: {e}")
    exit()



Training Random Forest with Grid Search...

Best Random Forest Model:
   - Best F1 Score: 0.6111
   - Best Parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 8, 'min_samples_split': 2, 'n_estimators': 500}


In [17]:


best_rf_model = grid_rf.best_estimator_

# Display available teams

teams = list(encoder.classes_)



print("Available Teams:")
for i, team in enumerate(teams, start=1):
    print(f"{i}: {team}")


try:
    home_team_index = int(input("Select the home team by number: ")) - 1
    away_team_index = int(input("Select the away team by number: ")) - 1

    home_team = teams[home_team_index]
    away_team = teams[away_team_index]

    print(f"\nHome Team: {home_team}")
    print(f"Away Team: {away_team}")

except (IndexError, ValueError):
    print("Invalid team selection. Please enter a valid number.")
    exit()




# Features selection

home_features = [feature for feature in selected_features if feature.startswith("Home ")]
away_features = [feature for feature in selected_features if feature.startswith("Away ")]


home_team_stats = results1[results1["Home Team"] == home_team][home_features].mean()
away_team_stats = results1[results1["Away Team"] == away_team][away_features].mean()

if home_team_stats.isnull().any() or away_team_stats.isnull().any():
    print("Insufficient season data for one or both teams.")
    exit()


numerical_features = np.hstack([
    home_team_stats.values,  
    away_team_stats.values   
])


num_teams_in_training = best_rf_model.n_features_in_ - len(numerical_features)
one_hot_home = [1 if team == home_team else 0 for team in teams][:num_teams_in_training // 2]
one_hot_away = [1 if team == away_team else 0 for team in teams][:num_teams_in_training // 2]


num_numerical_features = len(numerical_features)
num_one_hot_features = len(one_hot_home) + len(one_hot_away)
expected_num_features = best_rf_model.n_features_in_

print(f"Numerical features: {num_numerical_features}")
print(f"One-hot encoded features: {num_one_hot_features}")
print(f"Total features before reshaping: {num_numerical_features + num_one_hot_features}")
print(f"Model expects: {expected_num_features} features")


if num_numerical_features + num_one_hot_features != expected_num_features:
    print("Feature mismatch detected! Exiting.")
    exit()





#Inputs

input_vector = np.hstack([
    numerical_features,
    one_hot_home,
    one_hot_away
]).reshape(1, -1) 


predicted_prob = best_rf_model.predict_proba(input_vector)


home_team_prob = predicted_prob[0][1]
away_team_prob = predicted_prob[0][0] 


try:
    home_team_odds = float(input(f"\nEnter the odds for {home_team}: "))
    away_team_odds = float(input(f"Enter the odds for {away_team}: "))
except ValueError:
    print("Invalid odds input. Please enter numeric values.")
    exit()


def kelly_criterion(prob, odds):
    b = odds - 1
    q = 1 - prob
    return max(0, (b * prob - q) / b)


home_kelly = kelly_criterion(home_team_prob, home_team_odds)
away_kelly = kelly_criterion(away_team_prob, away_team_odds)


print("\nBetting Recommendation:")
if home_kelly > away_kelly and home_kelly > 0:
    print(f"Bet on {home_team}. Kelly Criterion suggests betting {home_kelly:.2%} of your bankroll.")
elif away_kelly > home_kelly and away_kelly > 0:
    print(f"Bet on {away_team}. Kelly Criterion suggests betting {away_kelly:.2%} of your bankroll.")
else:
    print("Neither bet has a positive expected value according to the Kelly Criterion.")


print("\nFinal Predictions:")
print(f"{home_team} Win Probability: {home_team_prob:.2%}, Odds: {home_team_odds}")
print(f"{away_team} Win Probability: {away_team_prob:.2%}, Odds: {away_team_odds}")
print(f"Kelly Criterion - {home_team}: {home_kelly:.2%}")
print(f"Kelly Criterion - {away_team}: {away_kelly:.2%}")

Available Teams:
1: Anaheim Ducks
2: Boston Bruins
3: Buffalo Sabres
4: Calgary Flames
5: Carolina Hurricanes
6: Chicago Blackhawks
7: Colorado Avalanche
8: Columbus Blue Jackets
9: Dallas Stars
10: Detroit Red Wings
11: Edmonton Oilers
12: Florida Panthers
13: Los Angeles Kings
14: Minnesota Wild
15: Montreal Canadiens
16: Nashville Predators
17: New Jersey Devils
18: New York Islanders
19: New York Rangers
20: Ottawa Senators
21: Philadelphia Flyers
22: Pittsburgh Penguins
23: San Jose Sharks
24: Seattle Kraken
25: St. Louis Blues
26: Tampa Bay Lightning
27: Toronto Maple Leafs
28: Utah Hockey Club
29: Vancouver Canucks
30: Vegas Golden Knights
31: Washington Capitals
32: Winnipeg Jets


Select the home team by number:  23
Select the away team by number:  2



Home Team: San Jose Sharks
Away Team: Boston Bruins
Numerical features: 10
One-hot encoded features: 64
Total features before reshaping: 74
Model expects: 74 features



Enter the odds for San Jose Sharks:  2
Enter the odds for Boston Bruins:  2



Betting Recommendation:
Bet on San Jose Sharks. Kelly Criterion suggests betting 73.05% of your bankroll.

Final Predictions:
San Jose Sharks Win Probability: 86.52%, Odds: 2.0
Boston Bruins Win Probability: 13.48%, Odds: 2.0
Kelly Criterion - San Jose Sharks: 73.05%
Kelly Criterion - Boston Bruins: 0.00%
